## Train Ranking Model

In this notebook, we will train a ranking model using gradient boosted trees. 

Let's start by loading the datasets we created in the previous notebook.

In [ ]:
# Uncomment this cell and fill in details if you are running external Python
import os
key=""
with open("api-key.txt", "r") as f:
    key = f.read().rstrip()
os.environ['HOPSWORKS_PROJECT']="hm"
os.environ['HOPSWORKS_HOST']="35.240.81.237"
os.environ['HOPSWORKS_API_KEY']=key

In [ ]:
import hopsworks

project = hopsworks.login()
fs = project.get_feature_store()
mr = project.get_model_registry()
dataset_api = project.get_dataset_api()

In [1]:
dataset_api.download("Resources/ranking_train.csv", overwrite=True)
dataset_api.download("Resources/ranking_validation.csv", overwrite=True)

Connected. Call `.close()` to terminate connection gracefully.


Downloading: 0.000%|          | 0/746313676 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/187122926 elapsed<00:00 remaining<?

'/srv/hops/staging/private_dirs/ffb1d102dffce5ff5c0f57ff37ec63cb45e08742c3ecb04de9e869c2a1bcdda4/ranking_validation.csv'

In [2]:
import pandas as pd

X_train = pd.read_csv("ranking_train.csv")
X_val = pd.read_csv("ranking_validation.csv")
y_train = X_train.pop("label")
y_val = X_val.pop("label")

X_train.sample(5)

,age,month_sin,month_cos,product_type_name,product_group_name,graphical_appearance_name,colour_group_name,perceived_colour_value_name,perceived_colour_master_name,department_name,index_name,index_group_name,section_name,garment_group_name
1359079,28.0,1.000000,6.123234e-17,Underwear Tights,Socks & Tights,Melange,Black,Dark,Black,Tights basic,Lingeries/Tights,Ladieswear,"Womens Nightwear, Socks & Tigh",Socks and Tights
1068568,25.0,0.866025,5.000000e-01,T-shirt,Garment Upper body,Melange,Dark Turquoise,Dark,Turquoise,Ladies Sport Bras,Sport,Sport,Ladies H&M Sport,Jersey Fancy
645030,30.0,-0.500000,8.660254e-01,Trousers,Garment Lower body,Denim,Blue,Medium Dusty,Blue,Trouser,Ladieswear,Ladieswear,Womens Everyday Collection,Trousers
359040,44.0,0.500000,-8.660254e-01,T-shirt,Garment Upper body,Solid,White,Light,White,Tops Fancy Jersey,Divided,Divided,Divided Collection,Jersey Fancy
3804484,26.0,0.500000,-8.660254e-01,Skirt,Garment Lower body,Solid,Black,Dark,Black,Skirts,Divided,Divided,Divided Collection,Skirts


In [4]:
X_val

,age,month_sin,month_cos,product_type_name,product_group_name,graphical_appearance_name,colour_group_name,perceived_colour_value_name,perceived_colour_master_name,department_name,index_name,index_group_name,section_name,garment_group_name
0,42.0,-8.660254e-01,5.000000e-01,Leggings/Tights,Garment Lower body,All over pattern,Black,Dark,Black,Jersey,Ladieswear,Ladieswear,Mama,Jersey Fancy
1,25.0,8.660254e-01,-5.000000e-01,Leggings/Tights,Garment Lower body,All over pattern,Black,Dark,Black,Jersey,Ladieswear,Ladieswear,Mama,Jersey Fancy
2,29.0,-8.660254e-01,5.000000e-01,Leggings/Tights,Garment Lower body,All over pattern,Black,Dark,Black,Jersey,Ladieswear,Ladieswear,Mama,Jersey Fancy
3,36.0,1.224647e-16,-1.000000e+00,Leggings/Tights,Garment Lower body,All over pattern,Black,Dark,Black,Jersey,Ladieswear,Ladieswear,Mama,Jersey Fancy
4,25.0,1.000000e+00,6.123234e-17,Leggings/Tights,Garment Lower body,All over pattern,Black,Dark,Black,Jersey,Ladieswear,Ladieswear,Mama,Jersey Fancy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1161606,41.0,-1.000000e+00,-1.836970e-16,Trousers,Garment Lower body,Solid,Greenish Khaki,Dark,Khaki green,Trouser,Menswear,Menswear,Contemporary Casual,Trousers
1161607,28.0,-8.660254e-01,-5.000000e-01,Trousers,Garment Lower body,Solid,Greenish Khaki,Dark,Khaki green,Trouser,Menswear,Menswear,Contemporary Casual,Trousers
1161608,39.0,-1.000000e+00,-1.836970e-16,Trousers,Garment Lower body,Solid,Greenish Khaki,Dark,Khaki green,Trouser,Menswear,Menswear,Contemporary Casual,Trousers
1161609,32.0,-5.000000e-01,8.660254e-01,Trousers,Garment Lower body,Solid,Greenish Khaki,Dark,Khaki green,Trouser,Menswear,Menswear,Contemporary Casual,Trousers


Let's train a model.

In [5]:
from catboost import CatBoostClassifier, Pool

cat_features = list(
    X_train.select_dtypes(include=['string', 'object']).columns
)

pool_train = Pool(X_train, y_train, cat_features=cat_features)
pool_val = Pool(X_val, y_val, cat_features=cat_features)

model = CatBoostClassifier(
    learning_rate=0.2,
    iterations=100,
    depth=10,
    scale_pos_weight=10,
    early_stopping_rounds=5,
    use_best_model=True
)

model.fit(pool_train, eval_set=pool_val)

0:	learn: 0.6871018	test: 0.6871234	best: 0.6871234 (0)	total: 3.42s	remaining: 5m 38s
1:	learn: 0.6870993	test: 0.6871230	best: 0.6871230 (1)	total: 6.39s	remaining: 5m 12s
2:	learn: 0.6820026	test: 0.6821280	best: 0.6821280 (2)	total: 8.88s	remaining: 4m 46s
3:	learn: 0.6820024	test: 0.6821280	best: 0.6821280 (2)	total: 10.2s	remaining: 4m 4s
4:	learn: 0.6798912	test: 0.6800121	best: 0.6800121 (4)	total: 12.8s	remaining: 4m 4s
5:	learn: 0.6773005	test: 0.6774476	best: 0.6774476 (5)	total: 15.2s	remaining: 3m 58s
6:	learn: 0.6760540	test: 0.6761763	best: 0.6761763 (6)	total: 17.7s	remaining: 3m 54s
7:	learn: 0.6729960	test: 0.6732147	best: 0.6732147 (7)	total: 19.8s	remaining: 3m 47s
8:	learn: 0.6717676	test: 0.6720242	best: 0.6720242 (8)	total: 22.2s	remaining: 3m 44s
9:	learn: 0.6707740	test: 0.6710661	best: 0.6710661 (9)	total: 24.4s	remaining: 3m 39s
10:	learn: 0.6707739	test: 0.6710661	best: 0.6710661 (9)	total: 25.2s	remaining: 3m 23s
11:	learn: 0.6701090	test: 0.6704078	best: 0

Next, we'll evaluate how well the model performs on the validation data.

In [6]:
from sklearn.metrics import classification_report, precision_recall_fscore_support

preds = model.predict(pool_val)

precision, recall, fscore, _ = precision_recall_fscore_support(y_val, preds, average="binary")

metrics = {
    "precision" : precision,
    "recall" : recall,
    "fscore" : fscore
}

print(classification_report(y_val, preds))

              precision    recall  f1-score   support

           0       0.94      0.57      0.71   1056010
           1       0.13      0.66      0.22    105601

    accuracy                           0.58   1161611
   macro avg       0.54      0.61      0.47   1161611
weighted avg       0.87      0.58      0.67   1161611



It can be seen that the model has a low F1-score on the positive class (higher is better). The performance could potentially be improved by adding more features to the dataset, e.g. image embeddings.

## Feature Importance

In [7]:
feat_to_score = {feature: score for feature, score in zip(
    X_train.columns, model.feature_importances_)}

feat_to_score = dict(
    sorted(
        feat_to_score.items(),
        key=lambda item: item[1],
        reverse=True
    )
)

feat_to_score


{'product_type_name': 16.91749433963557,
 'month_sin': 15.430998232859832,
 'month_cos': 13.483437554124208,
 'product_group_name': 10.211329951925647,
 'age': 7.430632160433846,
 'department_name': 6.651642113420837,
 'index_name': 5.975907912052409,
 'perceived_colour_value_name': 5.005265027047932,
 'garment_group_name': 4.7903496666713306,
 'section_name': 4.488864459281177,
 'perceived_colour_master_name': 3.4300572290272866,
 'graphical_appearance_name': 3.1702418433090775,
 'index_group_name': 2.0446945188150845,
 'colour_group_name': 0.9690849913957855}

It can be seen that the model places high importance on user and item embedding features. Consequently, better trained embeddings could yield a better ranking model.

Finally, we'll save our model.

In [8]:
import joblib

joblib.dump(model, 'ranking_model.pkl')

['ranking_model.pkl']

### Upload Model to Model Registry

We'll upload the model to the Hopsworks Model Registry.

In [9]:
import hsml

# connect to Hopsworks Model Registry
conn = hsml.connection()
mr = conn.get_model_registry()

Connected. Call `.close()` to terminate connection gracefully.


In [10]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_example = X_train.sample().to_dict("records")
input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema, output_schema)

ranking_model = mr.python.create_model(
    name="ranking_model", metrics=metrics,
    model_schema=model_schema,
    input_example=input_example, description="Ranking model")

ranking_model.save("ranking_model.pkl")

  0%|          | 0/6 [00:00<?, ?it/s]

Exported model ranking_model with version 1


Model(name: 'ranking_model', version: 1)

### Next Steps

Now we have trained both a retrieval and a ranking model, which will allow us to generate recommendations for users. In the next notebook, we'll take a look at how we can deploy these models with the `HSML` library.